<U>Pure-pursuit Control</U> is a path tracking algorithm that makes the robot to follow a given trajectory. It simply computes the angular velocity command that moves the robot from its current position to reach some look-ahead point on the given trajectory. The robot then moves the nearest look-ahead point on the path based on the current position of the robot until the last point of the path. 

Given the pose (position and orientation) of the robot as an input, the object can be used to calculate the linear and angular velocities commands for the robot. 

The robot’s pose is input as a pose and orientation (theta) list of points as [x y theta]. The theta value is the angular orientation of the robot measured counterclockwise in radians from the x-axis (robot currently at 0 radians).



<img src='./figs/coord.png' width='400'>
<img src='./figs/lfc.png' width='400'>
<img src='./figs/lfc2.png' width='400'>

In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt

%matplotlib qt

<img src='./figs/car.png' width='300'>

In [3]:
class mobile():
    def __init__(self, x=0., y=0., yaw=0., v=0., \
                       l=2.9, lfc=1., dt=0.1):
        """
        x, y: 2D position
        v: velocity
        yaw: orientation according to z-axis
        l: wheel base [m]
        lfc: look-ahead distance
        dt: [s]
        """
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.l = l
        self.lfc = lfc
        self.dt = dt
        
    def update(self, a, delta):
        self.x += self.v*math.cos(self.yaw)*self.dt
        self.y += self.v*math.sin(self.yaw)*self.dt
        self.yaw += self.v/self.l*math.tan(delta)*self.dt
        self.v += a*self.dt  

In [8]:
def get_idx(robot, x_ref, y_ref):
    """
    get nearest position index
    """
    dx = [robot.x - x_i for x_i in x_ref]
    dy = [robot.x - y_i for y_i in y_ref]
    d = [abs(math.sqrt(dx_i**2 + dy_i**2)) for (dx_i, dy_i) in zip(dx, dy)]
    idx = d.index(min(d))

    lfc_i = 0
    while robot.lfc > lfc_i and (idx + 1) < len(x_ref):
        dx = x_ref[idx + 1] - x_ref[idx]
        dy = y_ref[idx + 1] - y_ref[idx]
        lfc_i += math.sqrt(dx**2 + dy**2)
        idx += 1
    
    return idx

In [9]:
def pure_pursuit_control(robot, x_ref, y_ref, idx_ref):
    idx = get_idx(robot, x_ref, y_ref)
    
    if idx_ref >= idx:
        idx = idx_ref
    
    if idx < len(x_ref):
        tx = x_ref[idx]
        ty = y_ref[idx]
    else:
        tx = x_ref[-1]
        ty = y_ref[-1]
        idx = len(x_ref) - 1
        
    theta = math.atan2(ty - robot.y, tx - robot.x) - robot.yaw
    
    if robot.v < 0: # backward
        theta = math.pi - theta
        
    delta = math.atan2(2.0*robot.l*math.sin(theta)/robot.lfc, 1.0)
    
    return delta, idx

In [13]:
anim = True

# define reference track and velocity
x_ref = np.arange(0, 50, 0.1)
y_ref = [math.sin(i/5.0)*i/2.0 for i in x_ref]
v_ref = 10./3.6 # [m/s]

t_T = 100.
t = 0

x_0 = 0.
y_0 = -3.
yaw_0 = 0.
v_0 = 0.
robot = mobile(x=x_0, y=y_0, yaw=yaw_0, v=v_0)

# save trajectories
x_list, y_list, yaw_list, v_list = [], [], [], []
t_list = []
x_list.append(robot.x)
y_list.append(robot.y)
yaw_list.append(robot.yaw)
v_list.append(robot.v)
t_list.append(0.0)
idx_ref = get_idx(robot, x_ref, y_ref) 
    
Kp = 1.0
while t_T >= t and len(x_ref) > idx_ref:
    print('\r>> time: {:.3f} [s]'.format(t), end='')
    a_i = Kp * (v_ref - robot.v)
    d_i, idx_ref = pure_pursuit_control(robot, x_ref, y_ref, idx_ref)
    
    robot.update(a_i, d_i)

    t += robot.dt
    
    x_list.append(robot.x)
    y_list.append(robot.y)
    yaw_list.append(robot.yaw)
    v_list.append(robot.v)
    t_list.append(0.0)    

    if anim:
        plt.cla()
        plt.plot(x_ref, y_ref, ".r", label="course")
        plt.plot(x_list, y_list, "-b", label="trajectory")
        plt.plot(x_ref[idx_ref], y_ref[idx_ref], "xg", label="target")
        plt.axis("equal")
        plt.grid(True)
        plt.title("Speed[km/h]:" + str(robot.v * 3.6)[:4])
        plt.pause(0.001)

>> time: 0.000>> Move look-ahead point
>> time: 0.100>> Move look-ahead point
>> time: 0.200>> Move look-ahead point
>> time: 0.300>> Move look-ahead point
>> time: 0.400>> Move look-ahead point
>> time: 0.500>> Move look-ahead point
>> time: 0.600>> Move look-ahead point
>> time: 0.700>> Move look-ahead point
>> time: 0.800>> Move look-ahead point
>> time: 0.900>> Move look-ahead point
>> time: 1.000>> Move look-ahead point
>> time: 1.100>> Move look-ahead point
>> time: 1.200>> Move look-ahead point
>> time: 1.300>> Move look-ahead point
>> time: 1.400>> Move look-ahead point
>> time: 1.500>> Move look-ahead point
>> time: 1.600>> Move look-ahead point
>> time: 1.700>> Move look-ahead point
>> time: 1.800>> Move look-ahead point
>> time: 1.900>> Move look-ahead point
>> time: 2.000>> Move look-ahead point
>> time: 2.100>> Move look-ahead point
>> time: 2.200>> Move look-ahead point
>> time: 2.300>> Move look-ahead point
>> time: 2.400>> Move look-ahead point
>> time: 2.500>> Move loo

>> time: 21.000>> Move look-ahead point
>> time: 21.100>> Move look-ahead point
>> time: 21.200>> Move look-ahead point
>> time: 21.300>> Move look-ahead point
>> time: 21.400>> Move look-ahead point
>> time: 21.500>> Move look-ahead point
>> time: 21.600>> Move look-ahead point
>> time: 21.700>> Move look-ahead point
>> time: 21.800>> Move look-ahead point
>> time: 21.900>> Move look-ahead point
>> time: 22.000>> Move look-ahead point
>> time: 22.100>> Move look-ahead point
>> time: 22.200>> Move look-ahead point
>> time: 22.300>> Move look-ahead point
>> time: 22.400>> Move look-ahead point
>> time: 22.500>> Move look-ahead point
>> time: 22.600>> Move look-ahead point
>> time: 22.700>> Move look-ahead point
>> time: 22.800>> Move look-ahead point
>> time: 22.900>> Move look-ahead point
>> time: 23.000>> Move look-ahead point
>> time: 23.100>> Move look-ahead point
>> time: 23.200>> Move look-ahead point


KeyboardInterrupt: 

In [ ]:
assert 